<a href="https://colab.research.google.com/github/kienphan283/demo_vectordb/blob/master/langchain_vectorDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
%pip install -qU langchain-pinecone pinecone-notebooks

In [ ]:
%%capture
%pip install datasets

In [ ]:
from datasets import load_dataset
data = load_dataset('squad', split='train')
data = data.to_pandas()[['id', 'context', 'title']]
data.drop_duplicates(subset='context', keep='first', inplace=True)
data.head()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,id,context,title
0,5733be284776f41900661182,"Architecturally, the school has a Catholic cha...",University_of_Notre_Dame
5,5733bf84d058e614000b61be,"As at most other universities, Notre Dame's st...",University_of_Notre_Dame
10,5733bed24776f41900661188,The university is the major seat of the Congre...,University_of_Notre_Dame
15,5733a6424776f41900660f51,The College of Engineering was established in ...,University_of_Notre_Dame
20,5733a70c4776f41900660f64,All of Notre Dame's undergraduate students are...,University_of_Notre_Dame


In [ ]:
import getpass
import os
import time

from pinecone import Pinecone, ServerlessSpec

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

Enter your Pinecone API key: ··········


In [ ]:
pc.delete_index("langchain-test-index")

In [ ]:
import time

index_name = "langchain-test-index"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [ ]:
from uuid import uuid4
from langchain_core.documents import Document

docs = []

for i, row in data.iterrows():
  docs.append(Document(
      page_content=row['context'], # Đặt page_content là nội dung từ cột 'context'
      metadata={
          'id': row['id'],
          'title': row['title']
      }
  ))

print(len(docs))
print(docs[0])

18891
page_content='Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.' metadata={'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame'}


In [ ]:
%%capture
%pip install -qU langchain-huggingface
%pip install -qU langchain-nomic

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
uuids = [str(uuid4()) for _ in range(len(docs))]

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu', 'trust_remote_code': True}
encode_kwargs = {'normalize_embeddings': False}

# Khởi tạo HuggingFaceEmbeddings với mô hình Nomic Embed và API key
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# Thêm tài liệu vào vector store
vector_store.add_documents(documents=docs, ids=uuids)

['63a82423-c137-4a7e-8979-41c3d01c119e',
 'f60eaa00-3fde-4142-a667-b8d607f31bd4',
 '0a9d0d64-26e4-437b-8a1b-2158ffff52ca',
 '710d4124-22a1-4c77-96ab-a29c97ccfa4c',
 '95c076f8-5ff6-4ae6-8c43-29d9b8161519',
 'b7afee83-8246-4ee1-8e24-dfb331426792',
 'ad39005f-c107-4e8c-ade8-2eea2e47d28b',
 'f68a10ff-012f-440a-87c8-fad8aa0c9ad1',
 'ec1944cb-1557-43c0-a257-9549d6919e7f',
 'b066cc38-11b7-46e1-82aa-e951093b9d56',
 'bc514469-ef98-4f07-9148-951638e4d22b',
 '4fe49715-e4b4-477a-ad44-ac29c643a48c',
 '2cb4a956-b740-4e24-a427-9c162694a1d2',
 'ff062849-f2d3-4f79-8a20-fd0aa2ea90c5',
 '4ca06807-5bed-4244-a02a-e585394e7b2b',
 'edeaaf1e-d913-4c52-acd4-3ee414b2645b',
 '68648e1c-fdbc-4adf-b206-df1c46e05ecb',
 '45c21b19-53b6-4948-bad0-fbe438384a2b',
 '965a3a9b-2fc9-4fe6-bbea-7587c1ad9948',
 '0cf900ba-0039-4f5b-8502-756e2e3e0cf3',
 '5206cfb0-2052-4912-8b61-999fc5e62b8a',
 '207b7667-8d06-41db-8f18-a4793683f16e',
 '9303f2f7-a6ae-404a-a184-e4e0f34c7123',
 '1813200d-4c8f-4a98-adf6-71f87ca1253c',
 'ba0afec1-6527-

In [ ]:
query = "What is the University of Notre Dame known for?"
query_embed = embeddings.embed_query(query)
query_embed

[0.002104654209688306,
 0.049631401896476746,
 0.04120035842061043,
 -0.034091535955667496,
 0.06217263638973236,
 -0.008577628061175346,
 -0.057920508086681366,
 -0.02610936015844345,
 0.032026257365942,
 0.02674485743045807,
 -0.023487813770771027,
 0.0011501979315653443,
 -0.0031053810380399227,
 -0.05043993517756462,
 -0.08068621903657913,
 -0.060198988765478134,
 -0.03596055507659912,
 -0.06692366302013397,
 0.09738291800022125,
 -0.06163080409169197,
 -0.03986454755067825,
 0.037880755960941315,
 0.01238209381699562,
 -0.0470595546066761,
 -0.0038671381771564484,
 -0.009042340330779552,
 0.043287016451358795,
 -0.08141108602285385,
 -0.06521748751401901,
 -0.03865709528326988,
 0.010561776347458363,
 0.02110528200864792,
 0.00589991407468915,
 0.02032090350985527,
 -0.08145730942487717,
 0.0017127839382737875,
 0.07609042525291443,
 -0.018083009868860245,
 0.004730451386421919,
 0.0478157177567482,
 -0.0696738064289093,
 0.00933748297393322,
 0.10209595412015915,
 0.0429086461663

In [ ]:
!pip install pinecone-client
!pip install -U langchain-community

from langchain.vectorstores import Pinecone

# Assuming 'index' is your Pinecone index object and 'embeddings' is your embedding model
vector_store = Pinecone(index, embeddings.embed_query, "text")

# Truy vấn vector store bằng query_embed
docs_and_scores = vector_store.similarity_search_by_vector(query_embed, k=3) # Hoặc một số k khác tùy ý

# In ra các tài liệu tương tự tìm được
for doc, score in docs_and_scores:
    print(f"Nội dung tài liệu: {doc.page_content}, Điểm tương đồng: {score}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.4 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 0.3.0. An updated version of the class exists in the langchain-pinecone package and should be used instead. To use it run `pip install -U langchain-pinecone` and import as `from langchain_pinecone import Pinecone`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


NotImplementedError: 